<a href="https://colab.research.google.com/github/Ashfagithub/CODSOFT/blob/main/Spam_SMS_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd

In [32]:
df = pd.read_csv("/content/spam.csv", encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [33]:
df = df[["v1","v2"]]

In [34]:
df.rename(columns={'v1': 'Spam or ham','v2':'Message'}, inplace=True)
df.head()

,Spam or ham,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [35]:
type(df)

pandas.core.frame.DataFrame

In [36]:
df.describe()

,Spam or ham,Message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Spam or ham  5572 non-null   object
 1   Message      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [38]:
df.drop_duplicates(inplace=True)

In [39]:
df['spam'] = df["Spam or ham"].apply(lambda x:1 if x=="spam" else 0)
df.head()

,Spam or ham,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.Message,df.spam,test_size=0.25)

*converting text data into numerical format based on the frequency of words.*

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

# Initializing CountVectorizer
v = CountVectorizer()

# Transforming the text data (X_train.values) into a matrix of token counts
X_train_count = v.fit_transform(X_train.values)

# Converting the matrix of token counts to a dense array representation (showing the first three rows)
X_train_count.toarray()[:3]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

*Build and Train Models using Naive Bayes classifiers*

In [43]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y_train)

MultinomialNB()

*Evaluation of model performance*

In [44]:
emails = [
    "Hey Ashfa, How are you",
    'Upto 20% discount on parking,exclusive offer just for you'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1])

In [45]:
X_test_count = v.transform(X_test)
model.score(X_test_count,y_test)

0.9868522815158546

In [47]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vetorizer', CountVectorizer()),
    ('nb',MultinomialNB())
])

In [48]:
clf.fit(X_train,y_train)

Pipeline(steps=[('vetorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [49]:
clf.predict(emails)

array([0, 1])

***implementation using scikit-learn with TF-IDF and a Naive Bayes classifier***

---



In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Convert text data to numerical features using TF-IDF
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Initialize and train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Accuracy: 0.9505027068832174
Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1126
           1       1.00      0.62      0.76       167

    accuracy                           0.95      1293
   macro avg       0.97      0.81      0.87      1293
weighted avg       0.95      0.95      0.95      1293

